In [1]:
import requests
import pandas as pd
import json
import sys
from pathlib import Path
import xlrd
import csv
import os
import tempfile

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [23]:
with open("config.json") as f:
    config = json.load(f)
    for key in config:
        os.environ[key] = config[key]

In [15]:
def xlsUrlToDF(url:str) -> pd.DataFrame:
    r = requests.get(url, allow_redirects=True)
    df = pd.DataFrame()
    with tempfile.NamedTemporaryFile("wb") as xls_file:
        xls_file.write(r.content)
    
        try:
            book = xlrd.open_workbook(xls_file.name,encoding_override="utf-8")  
        except:
            book = xlrd.open_workbook(xls_file.name,encoding_override="cp1251")

        sh = book.sheet_by_index(0)
        with tempfile.NamedTemporaryFile("w") as csv_file:
            wr = csv.writer(csv_file, quoting=csv.QUOTE_ALL)

            for rownum in range(sh.nrows):
                wr.writerow(sh.row_values(rownum))
            df = pd.read_csv(csv_file.name)
            csv_file.close()

        xls_file.close()
    return df

In [24]:
# TODO url to config

def api(query,from_study,to_study):
    url = os.environ["URL_USA"].format(query,from_study,to_study)
    response = requests.request("GET", url)
    return response.json()

In [25]:
def apiWrapper(query,from_study):
    return api(query,from_study,from_study+99)

In [26]:
def getAllStudiesByQuery(query:str) -> list:
    studies:list = []
    from_study = 1
    temp = apiWrapper(query,from_study)
    nstudies = temp['FullStudiesResponse']['NStudiesFound']
    print("> {} studies found by '{}' keyword".format(nstudies,query))
    if nstudies > 0:
        studies = temp['FullStudiesResponse']['FullStudies']
        for study_index in range(from_study+100,nstudies,100):
            temp = apiWrapper(query,study_index)
            studies.extend(temp['FullStudiesResponse']['FullStudies'])
    
    return studies

In [29]:
#TODO to config
url_int = os.environ["URL_INT"]

In [30]:
internationalstudies = xlsUrlToDF(url_int)

In [31]:
internationalstudies

,TrialID,Last Refreshed on,Public title,Scientific title,Acronym,Primary sponsor,Date registration,Date registration3,Export date,Source Register,...,Condition,Intervention,Primary outcome,results date posted,results date completed,results url link,Retrospective flag,Bridging flag truefalse,Bridged type,results yes no
0,ChiCTR2000029953,43878.0,Construction and Analysis of Prognostic Predic...,Construction and Analysis of Prognostic Predic...,NaN,Zhongnan Hospital of Wuhan University,43878.0,20200217.0,43834.660579,ChiCTR,...,2019-nCoV Pneumonia,survival group:none;died:none;,duration of in hospital;in hospital mortality;...,NaN,NaN,NaN,No,0,,NaN
1,ChiCTR2000029935,43878.0,A Single-arm Clinical Trial for Chloroquine Ph...,A Single-arm Clinical Trial for Chloroquine Ph...,NaN,"HwaMei Hospital, University of Chinese Academy...",43877.0,20200216.0,43834.660579,ChiCTR,...,Novel Coronavirus Pneumonia (COVID-19),Case series:Treated with conventional treatmen...,Length of hospital stay;,NaN,NaN,NaN,No,0,,NaN
2,ChiCTR2000029850,43878.0,Study on convalescent plasma treatment for sev...,Effecacy and safty of convalescent plasma trea...,NaN,The First Affiliated Hospital of Zhejiang Univ...,43876.0,20200215.0,43834.660579,ChiCTR,...,Novel Coronavirus Pneumonia (COVID-19),experimental group:standardized comprehensive ...,Fatality rate;,NaN,NaN,NaN,Yes,0,,NaN
3,ChiCTR2000029814,43878.0,Clinical Trial for Integrated Chinese and West...,Clinical Trial for Integrated Chinese and West...,NaN,Children's Hospital of Fudan University,43875.0,20200214.0,43834.660579,ChiCTR,...,Novel Coronavirus Pneumonia (COVID-19),control group:Western Medicine;experimental gr...,Time fo fever reduction;Time of nucleic acid n...,NaN,NaN,NaN,Yes,0,,NaN
4,NCT04245631,43878.0,"Development of a Simple, Fast and Portable Rec...","Development of a Simple, Fast and Portable Rec...",NaN,Beijing Ditan Hospital,43856.0,20200126.0,43834.660579,ClinicalTrials.gov,...,New Coronavirus,Diagnostic Test: Recombinase aided amplificati...,Detection sensitivity is greater than 95%;Dete...,NaN,NaN,NaN,Yes,0,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
774,ChiCTR2000031204,43920.0,"A multicenter, single-blind, randomized contro...","A multicenter, single-blind, randomized contro...",NaN,"Beijing you'an Hospital, Capital Medical Unive...",43914.0,20200324.0,43834.660579,ChiCTR,...,Novel Coronavirus Pneumonia (COVID-19),Experimental treatment group:Oral chloroquine ...,Clearance time of virus RNA;,NaN,NaN,NaN,No,0,,NaN
775,ChiCTR2000031187,43920.0,A medical records based retrospective study fo...,A medical records based retrospective study fo...,NaN,Wuhan Third Hospital & Tongren Hospital of Wuh...,43913.0,20200323.0,43834.660579,ChiCTR,...,Novel Coronavirus Pneumonia (COVID-19),Case series:?;,PT-PCR test;,NaN,NaN,NaN,Yes,0,,NaN
776,ChiCTR2000030936,43920.0,A real-world study for the Chinese medicines '...,Hospital of Chengdu University of Traditional ...,NaN,Hospital of Chengdu University of Traditional ...,43908.0,20200318.0,43834.660579,ChiCTR,...,Novel Coronavirus Pneumonia (COVID-19),CM group:Xinguan No. 2 / Xinguan No. 3 alone o...,Body temperature returns to normal time;After ...,NaN,NaN,NaN,No,0,,NaN
777,ChiCTR2000030923,43920.0,The treatment and diagnosis plan of integrated...,The treatment and diagnosis plan of integrated...,NaN,Affiliated Hospital of Shaanxi University of T...,43907.0,20200317.0,43834.660579,ChiCTR,...,Novel Coronavirus Pneumonia (COVID-19),Suspected case treatment group:TCM formula 1 o...,Incidence of COVID-19 pneumonia;COVID-19 pneum...,NaN,NaN,NaN,No,0,,NaN


In [32]:
apiWrapper("covid-19",1)

{'FullStudiesResponse': {'APIVrs': '1.01.02',
  'DataVrs': '2020:04:03 00:51:56.804',
  'Expression': 'covid-19',
  'NStudiesAvail': 335064,
  'NStudiesFound': 318,
  'MinRank': 1,
  'MaxRank': 100,
  'NStudiesReturned': 100,
  'FullStudies': [{'Rank': 1,
    'Study': {'ProtocolSection': {'IdentificationModule': {'NCTId': 'NCT04330261',
       'OrgStudyIdInfo': {'OrgStudyId': 'REB18-0107_MOD9'},
       'Organization': {'OrgFullName': 'University of Calgary',
        'OrgClass': 'OTHER'},
       'BriefTitle': 'Clinical Characteristics and Outcomes of Pediatric COVID-19',
       'OfficialTitle': 'Clinical Characteristics and Outcomes of Children Potentially Infected by Severe Acute Respiratory Distress Syndrome (SARS)-CoV-2 Presenting to Pediatric Emergency Departments',
       'Acronym': 'PERN-COVID-19'},
      'StatusModule': {'StatusVerifiedDate': 'March 2020',
       'OverallStatus': 'Recruiting',
       'ExpandedAccessInfo': {'HasExpandedAccess': 'No'},
       'StartDateStruct': {'S